## Tools
https://langchain.readthedocs.io/en/latest/getting_started/getting_started.html

In [1]:
from dotenv import load_dotenv

# Load the environment variables from .env
load_dotenv()

True

### Load predefined tools

In [7]:
! pip install google-search-results


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Load in some tools to use
from langchain.agents import load_tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)


In [16]:
for tool in tools:
    print(tool)

name='Search' description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.' func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='0e06fba99dfce09b1ef7b14736248a801e4b6da97d58aeea528843c5b8ccb5d1', aiosession=None)> coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='0e06fba99dfce09b1ef7b14736248a801e4b6da97d58aeea528843c5b8ccb5d1', aiosession=None)>
name='Calculator' description='Useful for when you need to answer questions about math.' func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a

In [20]:
tools[0].invoke("Que hay para hacer en medellín? 3 sitios")

[{'position': 1,
  'block_position': 'top',
  'title': 'Medellin: Guatapé Tour, Lunch, Cruise, & Piedra del Peñol',
  'price': '$30',
  'extracted_price': 30,
  'link': 'https://www.getyourguide.com/',
  'source': 'GetYourGuide',
  'rating': 4.8,
  'reviews': 1400,
  'reviews_original': '1.4K',
  'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSgbmk5uY-QgJ_sL6BAxnsvG09XG01fXeZcwj7caXldT9Ygjg&s=4',
  'extensions': ['Free cancellation']},
 {'position': 2,
  'block_position': 'top',
  'title': 'Medellin: Full Day Guided City Highlights Tour',
  'price': '$66',
  'extracted_price': 66,
  'link': 'https://www.getyourguide.com/',
  'source': 'GetYourGuide',
  'rating': 4.8,
  'reviews': 114,
  'reviews_original': '114',
  'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjUimi4uBQZpwJrVPSmMwf07qmxIB9lAcwUjp3lFfnS_Mhyw&s=4',
  'extensions': ['Free cancellation']},
 {'position': 3,
  'block_position': 'top',
  'title': 'Medellin: Coffee Tour and Wellness 

In [21]:
tools[1].invoke("Que hay para hacer en medellín? 3 sitios")

ValueError: unknown format from LLM: The provided question is not a math problem, so I cannot answer it using Python's numexpr library.

In [22]:
llm_with_tools = llm.bind_tools(tools)
response1 = llm_with_tools.invoke("Quién es el actual presidente de Japón?")
print(response1.content) #Vacío
print(response1.tool_calls) # La invocación que requiere


[{'name': 'Search', 'args': {'__arg1': 'Who is the current president of Japan?'}, 'id': '063fefee-a96d-4e23-ad21-798be3d0df9b', 'type': 'tool_call'}]


In [24]:
response2 = llm_with_tools.invoke("hola")
print(response2.content) #Respuesta
print(response2.tool_calls) # vacio

I cannot fulfill this request. I do not have the functionality to greet users.
[]


### Create an Agent - Traditional method (initialize_agent)

In [10]:
# Finally, let's initialize an agent with:
# 1. The tools
# 2. The language model
# 3. The type of agent we want to use.
from langchain.agents import initialize_agent #Forma tradicional
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

d:\code\python\SETI\GenAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


See list of agents types [here](https://python.langchain.com/docs/modules/agents/agent_types/)

In [38]:
# Now let's test it out!
agent.run("Quíen es el presidente de Bolivia? y cual es el mayor número par menor a su edad?")



> Entering new AgentExecutor chain...
I need to search about the president of Bolivia and calculate the largest even number smaller than his age.
Action: Search
Action Input: presidente de Bolivia
Observation: Luis Arce Catacora
Thought:The president of Bolivia is Luis Arce Catacora. Let us calculate the largest even number smaller than his age.
Action: Search
Action Input: Luis Arce Catacora edad
Observation: 60 years
Thought:The president of Bolivia is 60 years old, so the largest even number smaller than his age is 58.
Final Answer: 58

> Finished chain.


'58'

### Create an Agent - LangGraph Prebuilt

In [28]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

agent_executor = create_react_agent(llm, tools)

In [36]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
for msg in response["messages"]:
    print(msg)

content='hi!' id='c1272e8c-10db-4421-b061-4691543eb828'
content='Hello there! I am a text-based AI assistant. How can I help you today?' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-db1527fe-ca4b-4b1a-b2e6-cad594ddfcda-0' usage_metadata={'input_tokens': 105, 'output_tokens': 19, 'total_tokens': 124}


In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Quíen es el presidente de Bolivia? y cual es el mayor número par menor a su edad?")]})


In [40]:
for msg in response["messages"]:
    print(msg)

content='Quíen es el presidente de Bolivia? y cual es el mayor número par menor a su edad?' id='66225332-c98a-4d3b-966a-7ee6dbb07e7e'
content='' additional_kwargs={'function_call': {'name': 'Search', 'arguments': '{"__arg1": "presidente de Bolivia"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-607a8520-bec9-4461-9183-12dcd23a4d24-0' tool_calls=[{'name': 'Search', 'args': {'__arg1': 'presidente de Bolivia'}, 'id': '7fd76503-ae15-4690-916b-bb75cb8dbcbf', 'type': 'tool_call'}] usage_metadata={'input_tokens': 124, 'output_tokens': 16, 

In [3]:
import os
"GOOGLE_API_KEY" in os.environ and "SERPAPI_API_KEY" in os.environ

True

# Building A Language Model Application
### LLMS: Get predictions from a language model

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
chat_groq = ChatGroq(temperature=0, model='llama3-groq-8b-8192-tool-use-preview')
llm = ChatGoogleGenerativeAI(temperature=0.9, model='gemini-pro')

d:\code\python\SETI\GenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
